In [1]:
import pandas as pd
import re
pd.options.mode.chained_assignment = None  # Remove copy in place warning for dataframe operations
#import jellyfish

# Définition du fichier d'alignement à vérifier
input_file_path = "input/P14_occ_moyennes_initiales.tsv"

# Définition du fichier de sortie pour les cas suspects
output_file_root = "output/P14_a-verifier2"

# Faut-il exporter tous les alignements vérifiés avec une colonne identifiant les cas suspects (True),
# ou exporter ces derniers à part (False)?
output_in_place = True

# Nom de la colonne ajoutée
output_colname = "validation auto"

# Définition des colonnes contenant les formes sources et cible à comparer
source_form_colname = "forme principale source"
target_form_colname = "forme principale cible"
align_type_colname = "décision d'alignement"
target_colname = "réservoir cible"
target_candidatenr_colname = "nombre de candidats"

# Types d'alignement à vérifier
valid_types = {'externe', 'manuel'}

# Le fichier à vérifier est chargé dans une dataframe
df = pd.read_csv(input_file_path, sep='\t', dtype = str)
print(df.shape)

# Pour la comparaison, on retire les types d'alignements ignorés ainsi que les non-alignements
df_filtered = df[(df[align_type_colname].isin(valid_types)) & (df[target_colname] == 'idref')]
df_rest = df[(~df[align_type_colname].isin(valid_types)) | (df[target_colname] != 'idref')]
print(df_filtered.shape)
print(df_rest.shape)
display(df_filtered.head())

(9395, 23)
(7103, 23)
(2292, 23)


,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,id cible,forme principale cible,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3
0,rnv-nz-auth-atc,981023280174602851,"Bauer, Axel W",NaN,NaN,NaN,NaN,externe,1,1.0,...,077176561,"Bauer, Axel 1955-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rnv-nz-auth-atc,981023280175902851,"Smits, Jan M., 1967-",NaN,NaN,NaN,NaN,externe,1,0.9625,...,033304610,"Smits, Jan M. 1967-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rnv-nz-auth-atc,981023280246402851,"Windekens, Albert J. van",NaN,NaN,NaN,NaN,externe,1,1.0,...,028361091,"Van Windekens, Albert J. 1915-1989",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rnv-nz-auth-atc,981023280313602851,"Beeby-Thompson, A",NaN,NaN,NaN,NaN,externe,0,NaN,...,147520681,"Thompson, Arthur Beeby",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rnv-nz-auth-atc,981023280363402851,"Spiridonakis, Basile G",NaN,NaN,NaN,NaN,externe,1,0.9545454545454545,...,030406331,"Spiridonakis, Basile G. historien",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
# Comparaison des dates, s'il y en a

if not(output_in_place):
    wrong_dates = []
    misc_errors = []
    

# Fonction de comparaison des dates
def date_compare(source_date,target_date):
    source_date = str(source_date)
    target_date = str(target_date)
    # Ignorer les points d'interrogation
    source_date = source_date.replace('?','')
    target_date = target_date.replace('?','')
    # Retirer le zéro en début de date
    if source_date.startswith('0'):
        source_date = source_date[1:0]
    if target_date.startswith('0'):
        target_date = target_date[1:0]
    # Si l'une des dates à comparer est vide, on passe
    if ((source_date == '') | (target_date == '')):
        return False
    # Si l'une des dates comporte des points, ne comparer que les chiffres entre eux
    if (('.' in source_date) | ('.' in target_date)):
        #print('Comparaison avec points: ' + source_date + ' et ' + target_date)
        for char in range(0,len(source_date)):
            if ((source_date[char] != '.') & (target_date[char] != '.') & (source_date[char] != target_date[char])):
                #print('Je pense que ' + source_date[char] + ' != ' + target_date[char])
                return True
        return False
    if (source_date != target_date):
        return True
    else:
        return False

# Expression régulière pour trouver les dates
date_pattern = r'(?:ca\.|fl\.)?([0-9\.]{4}|\?)-?([0-9\.]{4}|\?)?'

df_filtered.reset_index(drop=True, inplace=True)  # Drop indexes since we're not going to use them

for index, row in df_filtered.iterrows():
    if not ((row[source_form_colname] == '') | (row[target_form_colname] == '') | pd.isnull(row[source_form_colname]) | pd.isnull(row[target_form_colname])):
        source_dates = re.findall(date_pattern, row[source_form_colname])
        target_dates = re.findall(date_pattern, row[target_form_colname])
        if ((len(source_dates) > 0) & (len(target_dates) > 0)):
            if (date_compare(source_dates[0][0],target_dates[0][0]) | date_compare(source_dates[0][1],target_dates[0][1])):
                # Cas potentiel de dates qui ne correspondent pas
                
                if (output_in_place):
                    df_filtered.loc[index, output_colname] = "Vérifier dates"
                else:
                    wrong_dates.append(row)
    else:
        # Il manque un des champs à comparer, autre erreur
        if (output_in_place):
            df_filtered.loc[index, output_colname] = "Erreur de validation"
        else:
            misc_errors.append(row)

if (output_in_place):
    print(df_filtered.shape)
    df_output = pd.concat([df_filtered,df_rest])
    print(df_output.shape)
    df_output.to_excel(output_file_root + "_tout.xlsx",index=False)
    display(df_filtered.head())
else:
    wrong_dates_df = pd.DataFrame(wrong_dates)
    misc_errors_df = pd.DataFrame(misc_errors)
        
    wrong_dates_df.to_excel(output_file_root + "_dates.xlsx",index=False)
    misc_errors_df.to_excel(output_file_root + "_erreurs.xlsx",index=False)


(7103, 24)
(9395, 24)


,réservoir source,id source,forme principale source,arbitre,date d'arbitrage,niveau de confiance,commentaire,décision d'alignement,nombre de candidats,score algo max,...,forme principale cible,type de cible 2,réservoir cible 2,id cible 2,forme principale cible 2,type de cible 3,réservoir cible 3,id cible 3,forme principale cible 3,validation auto
0,rnv-nz-auth-atc,981023280174602851,"Bauer, Axel W",NaN,NaN,NaN,NaN,externe,1,1.0,...,"Bauer, Axel 1955-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,rnv-nz-auth-atc,981023280175902851,"Smits, Jan M., 1967-",NaN,NaN,NaN,NaN,externe,1,0.9625,...,"Smits, Jan M. 1967-....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,rnv-nz-auth-atc,981023280246402851,"Windekens, Albert J. van",NaN,NaN,NaN,NaN,externe,1,1.0,...,"Van Windekens, Albert J. 1915-1989",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,rnv-nz-auth-atc,981023280313602851,"Beeby-Thompson, A",NaN,NaN,NaN,NaN,externe,0,NaN,...,"Thompson, Arthur Beeby",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,rnv-nz-auth-atc,981023280363402851,"Spiridonakis, Basile G",NaN,NaN,NaN,NaN,externe,1,0.9545454545454545,...,"Spiridonakis, Basile G. historien",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
